# Hospital Management Data Analysis

**Author:** Rakesh Kumar Gupta  
**Date:** 20 Aug 2025

This notebook loads raw hospital datasets, performs KPI calculations, and visualizes key insights.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

data_dir = Path("../data")
appointments = pd.read_csv(data_dir / "appointments.csv", parse_dates=["appointment_date"])
billing = pd.read_csv(data_dir / "billing.csv")
doctors = pd.read_csv(data_dir / "doctors.csv")
patients = pd.read_csv(data_dir / "patients.csv", parse_dates=["registration_date"])
treatments = pd.read_csv(data_dir / "treatments.csv")

# Basic shapes
print("Shapes:", { 
    "appointments": appointments.shape, 
    "billing": billing.shape, 
    "doctors": doctors.shape, 
    "patients": patients.shape, 
    "treatments": treatments.shape
})
appointments.head()


## KPI Calculations

In [ ]:

# Total revenue
total_revenue = billing["amount"].sum()

# Appointment rates
total_appts = len(appointments)
cancellation_rate = (appointments["status"].eq("Cancelled").sum() / total_appts) * 100
no_show_rate = (appointments["status"].eq("No-show").sum() / total_appts) * 100
completed_rate = (appointments["status"].eq("Completed").sum() / total_appts) * 100

# Join to compute branch revenue (completed only)
df_join = (treatments.merge(appointments, on="appointment_id", how="left")
                    .merge(billing, on="treatment_id", how="left")
                    .merge(doctors[["doctor_id", "hospital_branch", "specialization"]], on="doctor_id", how="left"))
branch_revenue = (df_join[df_join["status"]=="Completed"]
                  .groupby("hospital_branch")["amount"]
                  .sum()
                  .sort_values(ascending=False))

# Specialization revenue
spec_revenue = (df_join.groupby("specialization")["amount"]
                .sum().sort_values(ascending=False))

# Pending vs failed vs paid
billing_summary = billing.groupby("payment_status")["amount"].agg(["count","sum"]).sort_values("sum", ascending=False)

# Registration to first appointment days (completed only)
first_appt = (appointments.sort_values("appointment_date")
              .groupby("patient_id").first().reset_index())
merged = first_appt.merge(patients[["patient_id","registration_date","gender"]], on="patient_id", how="left")
merged = merged.merge(appointments[["patient_id","status","appointment_date"]], on=["patient_id","appointment_date"], how="left")
merged = merged[merged["status"].eq("Completed")]
merged["days_to_first"] = (merged["appointment_date"] - merged["registration_date"]).dt.days
avg_days_to_first = merged["days_to_first"].mean()

print({
    "Total Revenue": float(total_revenue),
    "Cancellation %": float(round(cancellation_rate,2)),
    "No-show %": float(round(no_show_rate,2)),
    "Completed %": float(round(completed_rate,2)),
    "Avg days registration->first completed appt": float(round(avg_days_to_first,2)) if not np.isnan(avg_days_to_first) else None
})

spec_revenue.head()


## Visualizations

In [ ]:

# Revenue by specialization
plt.figure()
spec_revenue.plot(kind="bar")
plt.title("Revenue by Specialization")
plt.xlabel("Specialization")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()


In [ ]:

# Branch revenue (completed appts)
plt.figure()
branch_revenue.plot(kind="bar")
plt.title("Branch Revenue (Completed Appointments)")
plt.xlabel("Hospital Branch")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()


In [ ]:

# Payment status distribution
plt.figure()
billing_summary["sum"].plot(kind="bar")
plt.title("Billing Amount by Payment Status")
plt.xlabel("Payment Status")
plt.ylabel("Total Amount")
plt.tight_layout()
plt.show()


In [ ]:

# Monthly trends (appointments & revenue) - if possible from appointment_date
# Approximate monthly revenue by joining billing to patient then to appointment month
appt_month = appointments.copy()
appt_month["month"] = appointments["appointment_date"].dt.to_period("M").astype(str)
# Attach billing via patient_id as per your SQL (note: this over-attributes if multiple bills per patient per month)
month_join = appt_month.merge(billing[["patient_id","amount"]], on="patient_id", how="left")
monthly = month_join.groupby("month").agg(total_appointments=("appointment_id","nunique"),
                                          total_revenue=("amount","sum")).fillna(0).reset_index()

# Plot appointments
plt.figure()
plt.plot(monthly["month"], monthly["total_appointments"])
plt.title("Monthly Appointments")
plt.xlabel("Month")
plt.ylabel("Appointments")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot revenue
plt.figure()
plt.plot(monthly["month"], monthly["total_revenue"])
plt.title("Monthly Revenue")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Save KPI summary to CSV

In [ ]:

out_dir = Path("../report")
out_dir.mkdir(parents=True, exist_ok=True)
kpis = {
    "total_revenue": [float(total_revenue)],
    "cancellation_rate_pct": [float(round(cancellation_rate,2))],
    "no_show_rate_pct": [float(round(no_show_rate,2))],
    "completed_rate_pct": [float(round(completed_rate,2))],
    "avg_days_to_first_completed": [float(round(avg_days_to_first,2)) if not np.isnan(avg_days_to_first) else None]
}
kpi_df = pd.DataFrame(kpis)
kpi_path = out_dir / "kpi_summary.csv"
kpi_df.to_csv(kpi_path, index=False)
kpi_path.as_posix()


## Notes
- This notebook mirrors the SQL logic used in the report.
- Charts use default matplotlib settings (no custom styles).